# Further Examples

## Linear Instrumental-Variables Regression
**Chapter 6** _Microeconometrics Using Stata_ by Cameron & Trivedi 

In [ ]:
import linearmodels as lm
from linearmodels import iv

In [ ]:
from linearmodels.datasets import meps
data = meps.load()
data = data.dropna()
print(meps.DESCR)

In [ ]:
data.columns

In [ ]:
controls = ['totchr', 'female', 'age', 'linc','blhisp']
print(data[['ldrugexp','hi_empunion'] + controls].describe(percentiles=[]))

In [ ]:
print(data[['ssiratio', 'lowincome', 'multlc', 'firmsz']].describe(percentiles=[]))

In [ ]:
from statsmodels.api import OLS, add_constant
data['const'] = 1
controls = ['const'] + controls

In [ ]:
import pandas as pd
ivolsmod = iv.IV2SLS(data.ldrugexp, data[['hi_empunion'] + controls], None, None)
res_ols = ivolsmod.fit()

In [ ]:
ivmod = iv.IV2SLS(data.ldrugexp, data[controls], data.hi_empunion, data.ssiratio)
res_2sls = ivmod.fit()
print(res_2sls.summary)

In [ ]:
# 2SLS with homoskedastic errorc
ivmod = iv.IV2SLS(data.ldrugexp, data[controls], data.hi_empunion, data[['ssiratio','multlc']])
res_2sls_std = ivmod.fit()
# 2SLS with heteroskedasticity robust SE
ivmod = iv.IV2SLS(data.ldrugexp, data[controls], data.hi_empunion, data[['ssiratio','multlc']])
res_2sls_robust = ivmod.fit(cov_type='robust')
# GMM
ivmod = iv.IVGMM(data.ldrugexp, data[controls], data.hi_empunion, data[['ssiratio','multlc']], weight_type='robust')
res_gmm = ivmod.fit()
# GMM with clustered standard errors by age
ivmod = iv.IVGMM(data.ldrugexp, data[controls], data.hi_empunion, data[['ssiratio','multlc']], weight_type='clustered', clusters=data.age)
res_gmm_clustered = ivmod.fit(cov_type='clustered', clusters=data.age)
# GMM - continuously updated
ivmod = iv.IVGMMCUE(data.ldrugexp, data[controls], data.hi_empunion, data[['ssiratio','multlc']])
res_gmm_cue = ivmod.fit(cov_type='robust')

In [ ]:
from collections import OrderedDict
from linearmodels.iv.results import compare
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
res['2SLS-Homo'] = res_2sls_std
res['2SLS-Hetero'] = res_2sls_robust
res['GMM'] = res_gmm
res['GMM Cluster(Age)'] = res_gmm_clustered
res['GMM-CUE'] = res_gmm_cue
compare(res)

In [ ]:
ivmod = iv.IV2SLS(data.ldrugexp, data[controls], data.hi_empunion, data.ssiratio)
res_2sls = ivmod.fit(cov_type='robust')
res_2sls.wu_hausman()

In [ ]:
step1 = iv.IV2SLS(data.hi_empunion, data[['ssiratio'] + controls], None, None).fit()
resids = step1.resids
exog = pd.concat([data[['hi_empunion'] + controls], resids],1)
step2 = iv.IV2SLS(data.ldrugexp, exog, None, None).fit(cov_type='robust')
print(step2.tstats.residual **2)

In [ ]:
res_gmm.j_stat

In [ ]:
all_instruments = ['ssiratio','lowincome','multlc','firmsz']
instruments = data[all_instruments]
ivmod = iv.IVGMM(data.ldrugexp, data[controls], data.hi_empunion, instruments, weight_type='robust')
res_gmm_all = ivmod.fit(cov_type='robust')
res_gmm_all.j_stat

In [ ]:
data[['hi_empunion'] + all_instruments].corr()

In [ ]:
od = OrderedDict()
for col in all_instruments:
    od[col] = iv.IV2SLS(data.ldrugexp, data[controls], data.hi_empunion, data[col]).fit(cov_type='robust')
od['OLS'] = res_ols
compare(od)


In [ ]:
res_2sls.first_stage

In [ ]:
ivmod = iv.IV2SLS(data.ldrugexp, data[controls], data.hi_empunion, instruments)
res_2sls_all = ivmod.fit(cov_type='robust')
res_2sls_all.first_stage

In [ ]:
ivmod = iv.IVLIML(data.ldrugexp, data[controls], data.hi_empunion, data[['ssiratio','multlc']])
res_liml = ivmod.fit(cov_type='robust')
compare({'2SLS':res_2sls_robust,'LIML': res_liml, 'GMM': res_gmm})

In [ ]:
print(res_liml.kappa)

### IV2SLS as OLS

In [ ]:
import pandas as pd
ivolsmod = iv.IV2SLS(data.ldrugexp, data[['hi_empunion'] + controls], None, None)
res_ivols = ivolsmod.fit()
sm_ols = res_ols.params
sm_ols.name = 'sm'
print(pd.concat([res_ivols.params, sm_ols], 1))